In [6]:
import cv2
import torch
from paddleocr import PaddleOCR
from ultralytics import YOLO
import os.path
import csv
import time
from datetime import datetime

In [8]:
# Load YOLOv8 trained model
model = YOLO("best.pt")  # Replace with your trained model path

# Initialize EasyOCR for text recognition
reader = PaddleOCR(use_angle_cls=True,lang='en')  # Supports multiple languages if needed

# Open video file
# video_path = "demo.mp4"
# cap = cv2.VideoCapture(video_path)

# Live cam
url = "http://192.168.238.92:8080/video"
cap = cv2.VideoCapture(url)

# Get video properties
width = int(cap.get(cv2.CAP_PROP_FRAME_WIDTH))
height = int(cap.get(cv2.CAP_PROP_FRAME_HEIGHT))
fps = int(cap.get(cv2.CAP_PROP_FPS))

# Define video writer
output_video = cv2.VideoWriter("output2.mp4", cv2.VideoWriter_fourcc(*"mp4v"), fps, (width, height))

# Threshold
THRESHOLD = 0.9

# Frame count to take 2 one frame per sec
FRAME_COUNT = 24
CUR_COUNT = 0

# Plate
plates = []
plate_d = dict()

# CSV file setup
csv_filename = "license_plates.csv"
if(not os.path.isfile(csv_filename)):
    with open(csv_filename, mode="w", newline="") as file:
        writer = csv.writer(file)
        writer.writerow(["License Plate", "TimeStamp"])

while cap.isOpened():
    ret, frame = cap.read()
    if not ret:
        break
    CUR_COUNT += 1
    if(CUR_COUNT < FRAME_COUNT):
        continue
    CUR_COUNT = 0

    # Perform YOLO detection
    results = model(frame)

    for result in results:
        boxes = result.boxes.xyxy  # Get bounding box coordinates
        for box in boxes:
            x1, y1, x2, y2 = map(int, box)
            plate_roi = frame[y1:y2, x1:x2]  # Crop number plate region

            # Perform OCR
            text_results = reader.ocr(plate_roi)
            # print(text_results)
            plate_text=""
            if text_results[0] is not None:
                for line in text_results[0]:
                    if(line[1][1]>=THRESHOLD):
                        plate_text+=line[1][0]
                print(plate_text)


            # Draw bounding box and text on frame
            if(plate_text != "" and len(plate_text)>=9):
                cv2.rectangle(frame, (x1, y1), (x2, y2), (0, 255, 0), 2)
                cv2.putText(frame, plate_text, (x1, y1 - 10), cv2.FONT_HERSHEY_SIMPLEX, 0.7, (0, 255, 0), 2)

                # Check if plate has detected 5 times
                if(plate_text not in plate_d.keys()):
                    plate_d[plate_text] = 1
                else:
                    plate_d[plate_text] += 1
                if(plate_d[plate_text] >= 2):
                    timestamp = datetime.now().strftime("%I:%M:%S %p on %B %d, %Y")
                    plate_d[plate_text] = 0
                    # Save to CSV
                    with open(csv_filename, mode="a", newline="") as file:
                        writer = csv.writer(file)
                        writer.writerow([plate_text, timestamp])

    # Save frame to video
    # output_video.write(frame)

    # Display video in real-time 
    
    cv2.imshow("ANPR", frame)
    if cv2.waitKey(1) & 0xFF == ord('q'):
        break
    

# Release resources
cap.release()
output_video.release()
cv2.destroyAllWindows()

[2025/02/15 10:51:34] ppocr DEBUG: Namespace(help='==SUPPRESS==', use_gpu=False, use_xpu=False, use_npu=False, use_mlu=False, ir_optim=True, use_tensorrt=False, min_subgraph_size=15, precision='fp32', gpu_mem=500, gpu_id=0, image_dir=None, page_num=0, det_algorithm='DB', det_model_dir='/home/jetson/.paddleocr/whl/det/en/en_PP-OCRv3_det_infer', det_limit_side_len=960, det_limit_type='max', det_box_type='quad', det_db_thresh=0.3, det_db_box_thresh=0.6, det_db_unclip_ratio=1.5, max_batch_size=10, use_dilation=False, det_db_score_mode='fast', det_east_score_thresh=0.8, det_east_cover_thresh=0.1, det_east_nms_thresh=0.2, det_sast_score_thresh=0.5, det_sast_nms_thresh=0.2, det_pse_thresh=0, det_pse_box_thresh=0.85, det_pse_min_area=16, det_pse_scale=1, scales=[8, 16, 32], alpha=1.0, beta=1.0, fourier_degree=5, rec_algorithm='SVTR_LCNet', rec_model_dir='/home/jetson/.paddleocr/whl/rec/en/en_PP-OCRv4_rec_infer', rec_image_inverse=True, rec_image_shape='3, 48, 320', rec_batch_num=6, max_text_le

[http @ 0xaaab044f1a10] Stream ends prematurely at 27422926, should be 18446744073709551615


In [5]:
f = open("detected_plates.txt","w+")
f.write("\n".join(plates))
f.close()

In [ ]:
l = [
    [[[[38.0, 16.0], [168.0, 8.0], [171.0, 50.0], [41.0, 58.0]], ('TN33B', 0.9214908480644226)], [[[10.0, 55.0], [31.0, 55.0], [31.0, 70.0], [10.0, 70.0]], ('NO', 0.8470568656921387)], [[[41.0, 60.0], [167.0, 52.0], [170.0, 94.0], [44.0, 102.0]], ('Y5979', 0.9254600405693054)]]
]
import pprint
# pprint.pprint(l[0][0])
text=""
for line in l[0]:
    if(line[1][1]>=0.9):
        text+=line[1][0]
print(text)

In [8]:
with open(csv_filename, mode="w", newline="") as file:
        writer = csv.writer(file)
        writer.writerow(["Timestamp", "License Plate"])